In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from io import StringIO
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import uuid
import os

# 셀레니움으로 번개장터 키워드 아이템 크롤링

In [52]:
# 웹 브라우저 열기
driver = webdriver.Chrome()
driver.maximize_window() 

In [ ]:
# URL로 이동
base_url = "https://m.bunjang.co.kr/"
driver.get(base_url)

## 키워드 검색

In [ ]:
# 1. 특정 키워드로 바로 이동
driver.get(base_url + "search/products?q=유모차 스토케&page=1")

In [ ]:
# 2. keyword 검색으로 이동
input_box_css = "#root > div > div > div.sc-dfVpRl.fxYqEy > div.sc-gzOgki.ciqSeg > div.sc-iyvyFf.kPWKwK > div.sc-jKJlTe.keyjxL > div.sc-eNQAEJ.voMyM > input"
input_box = driver.find_element(By.CSS_SELECTOR, input_box_css)
input_box.send_keys(" ")
input_box.send_keys("유모차 스토케")
input_box.send_keys(Keys.ENTER)

## 각 페이지별 아이템 리스트 크롤링

In [ ]:
# 모든 아이템 갯수 가져오기
import re

item_count_css = "#root > div > div > div:nth-child(4) > div > div.sc-hRmvpr.jtLTMQ > div > div.sc-cZBZkQ.ckPglo > span.sc-ecaExY.jPSzJz"
item_count = driver.find_element(By.CSS_SELECTOR, item_count_css).text
item_count = int(re.sub("[^0-9]", "", item_count))
item_count

In [ ]:
# 상품 href 리스트 만들기
item_card_css = "#root > div > div > div:nth-child(4) > div > div.sc-gacfCG.QBPXM > div > div > a"
item_card_list = driver.find_elements(By.CSS_SELECTOR, item_card_css)
len(item_card_list)

In [ ]:
# 페이지 수 세기
import math

page_count = 1
if len(item_card_list) < item_count:
    page_count = math.ceil(item_count / len(item_card_list))
page_count

In [ ]:
# 광고인지 체크
is_ad_css = "div.sc-gVyKpa.kXoKNq > span.sc-cpmKsF.buJhvy"
try:
    el = item_card_list[-15].find_element(By.CSS_SELECTOR, is_ad_css)
    is_ad = el.text == "광고"
except NoSuchElementException:
    is_ad = False

is_ad

In [ ]:
# 판매 완료
is_completed_css = "div > div > div > img"
try:
    el = item_card_list[0].find_element(By.CSS_SELECTOR, is_completed_css)
    is_completed = el.get_attribute('alt') == "판매 완료"
except NoSuchElementException:
    is_completed = False

is_completed

In [ ]:
from datetime import datetime, timedelta
import re

def parse_relative_time(text: str, now: datetime | None = None) -> datetime:
    """
    한국어 상대 시간을 실제 datetime으로 변환
    ex) "1분 전", "2시간 전", "3일 전", "3달 전"
    """
    if now is None:
        now = datetime.now()

    match = re.match(r"(\d+)(분|시간|일|달) 전", text.strip())
    if not match:
        raise ValueError(f"지원하지 않는 형식: {text}")

    value = int(match.group(1))
    unit = match.group(2)

    if unit == "분":
        return now - timedelta(minutes=value)
    elif unit == "시간":
        return now - timedelta(hours=value)
    elif unit == "일":
        return now - timedelta(days=value)
    elif unit == "달":
        # "달" → 30일로 가정
        return now - timedelta(days=value * 30)
    else:
        raise ValueError(f"알 수 없는 단위: {unit}")


# 사용 예시
now = datetime.now()
print(parse_relative_time("1분 전", now))   # → 2025-08-19 15:29:00
print(parse_relative_time("2시간 전", now)) # → 2025-08-19 13:30:00
print(parse_relative_time("3일 전", now))   # → 2025-08-16 15:30:00
print(parse_relative_time("3달 전", now))   # → 2025-05-21 15:30:00

In [ ]:
# 업로드 날짜
# 1분 전, 2시간 전, 3일 전, 3달 전

uploaded_date_css = "div.sc-iGPElx.iPIFfd > div.sc-gtfDJT.hZQMHB > div.sc-fOICqy.bgZehm"
uploaded_date = item_card_list[0].find_element(By.CSS_SELECTOR, uploaded_date_css)
uploaded_date = parse_relative_time(uploaded_date.text)

## 아이템 상세 정보 크롤링

In [ ]:
driver.get(item_card_list[0].get_attribute("href"))

In [ ]:
# 타이틀
title_css = "#root > div > div > div.Productsstyle__Wrapper-sc-13cvfvh-0.eVEUVR > div.Productsstyle__ProductPageTop-sc-13cvfvh-1.WbLlq > div > div.Productsstyle__ProductContentWrapper-sc-13cvfvh-8.jGywBa > div > div.Productsstyle__ProductSummaryWrapper-sc-13cvfvh-11.iDkwQU > div > div:nth-child(1) > div.ProductSummarystyle__Basic-sc-oxz0oy-2.ifrXrN > div.ProductSummarystyle__Name-sc-oxz0oy-3.dZBHcg"
title = driver.find_element(By.CSS_SELECTOR, title_css).text

# 가격
price_css = "#root > div > div > div.Productsstyle__Wrapper-sc-13cvfvh-0.eVEUVR > div.Productsstyle__ProductPageTop-sc-13cvfvh-1.WbLlq > div > div.Productsstyle__ProductContentWrapper-sc-13cvfvh-8.jGywBa > div > div.Productsstyle__ProductSummaryWrapper-sc-13cvfvh-11.iDkwQU > div > div:nth-child(1) > div.ProductSummarystyle__Basic-sc-oxz0oy-2.ifrXrN > div.ProductSummarystyle__PriceWrapper-sc-oxz0oy-4.dTIDFF > div"
price = driver.find_element(By.CSS_SELECTOR, price_css).text
price = re.sub("[^0-9]", "", price)

# 상품 상태
condition_css = "#root > div > div > div.Productsstyle__Wrapper-sc-13cvfvh-0.eVEUVR > div.Productsstyle__ProductPageTop-sc-13cvfvh-1.WbLlq > div > div.Productsstyle__ProductContentWrapper-sc-13cvfvh-8.jGywBa > div > div.Productsstyle__ProductSummaryWrapper-sc-13cvfvh-11.iDkwQU > div > div:nth-child(1) > div:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div.ProductSummarystyle__Value-sc-oxz0oy-21.eLyjky"
condition = driver.find_element(By.CSS_SELECTOR, condition_css).text

# 상품 정보
detail_css = "#root > div > div > div.Productsstyle__Wrapper-sc-13cvfvh-0.eVEUVR > div.Productsstyle__ProductPageTop-sc-13cvfvh-1.WbLlq > div > div.Productsstyle__ProductBottom-sc-13cvfvh-14.fxuPQD > div.Productsstyle__ProductInfoContent-sc-13cvfvh-13.jzEavb > div > div.ProductInfostyle__Wrapper-sc-ql55c8-0.gPJVxW > div.ProductInfostyle__Description-sc-ql55c8-2.hWujk > div.ProductInfostyle__DescriptionContent-sc-ql55c8-3.eJCiaL > p"
detail = driver.find_element(By.CSS_SELECTOR, detail_css).text


In [ ]:
import requests

def download_image(url: str, save_path: str, chunk_size: int = 1024) -> bool:
    """
    주어진 URL에서 이미지를 다운로드하여 지정한 경로에 저장합니다.

    Args:
        url (str): 이미지 URL
        save_path (str): 저장할 파일 경로
        chunk_size (int): 다운로드할 때 사용할 청크 크기 (기본 1024)

    Returns:
        bool: 성공 시 True, 실패 시 False
    """
    try:
        resp = requests.get(url, stream=True, timeout=10)
        if resp.status_code == 200:
            with open(save_path, "wb") as f:
                for chunk in resp.iter_content(chunk_size):
                    if chunk:  # keep-alive chunks 건너뛰기
                        f.write(chunk)
            print(f"다운로드 완료: {save_path}")
            return True
        else:
            print(f"다운로드 실패: HTTP {resp.status_code}")
            return False
    except Exception as e:
        print(f"오류 발생: {e}")
        return False


# 사용 예시
url = "https://media.bunjang.co.kr/product/345010693_1_1754091876_w480.jpg"
download_image(url, "bungae_image.jpg")


In [ ]:
# 이미지
image_list_css = "#root > div > div > div.Productsstyle__Wrapper-sc-13cvfvh-0.eVEUVR > div.Productsstyle__ProductPageTop-sc-13cvfvh-1.WbLlq > div > div.Productsstyle__ProductContentWrapper-sc-13cvfvh-8.jGywBa > div > div.Productsstyle__ProductImageWrapper-sc-13cvfvh-10.cXRuyi > div > div.sc-kLIISr.gWGEJy > div > img"
image_list = driver.find_elements(By.CSS_SELECTOR, image_list_css)

uid = uuid.uuid4()
image_path = "../../data/raw/" + str(uid)
os.makedirs(image_path, exist_ok=True)

for index, image in enumerate(image_list):
    url = image.get_attribute('src')
    download_image(url, f"{image_path}/bungaejangter_{uid}_{index}.jpg")

In [ ]:
# 모든 정보 
item = {
    "title" : title,
    "detail" : detail,
    "condition" : condition,
    "uploaded_date" : uploaded_date,
    "is_completed" : is_completed,
    "price": price,
}
items = []
items.append(item)
print(item)

In [ ]:
df = pd.DataFrame(items)

In [ ]:
df.to_csv("C:\Potenup\SecondHanded-Strollers-PredictedPrice\data\\raw\\bungaejangter.csv")

# 최종 크롤링 코드

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from io import StringIO
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from dataclasses import dataclass, asdict
import os
import csv
from item_crawler import get_item_data, get_keyword_item_count, get_keyword_page_count, get_page_keyword_item_list
from item_crawler import base_url, data_base_path, image_base_path

In [ ]:
import time

# 웹 브라우저 열기
driver = webdriver.Chrome()
driver.maximize_window() 

time.sleep(2)

In [ ]:
keyword_list = ["유모차 스토케"]

for keyword in keyword_list:
    # 키워드 URL로 이동
    print("============================")
    driver.get(base_url + f"search/products?q={keyword}&page=1")
    print("키워드: ", keyword)

    # 전체 상품 갯수
    item_count = get_keyword_item_count(driver, keyword)
    print("전체 아이템 수: ", item_count)
    page_count = get_keyword_page_count(driver, keyword, item_count)
    print("페이지수: ", page_count)    

    # 모든 아이템
    item_data = []
    for i in range(page_count) :
        page_item_list = get_page_keyword_item_list(driver, keyword, i + 1)
        print(f'페이지 {i + 1}: {len(page_item_list)}')

        for index, item in enumerate(page_item_list):
            link = item['link']
            if item['is_completed']:
                link += "&original=1"
            print(f"페이지 {i + 1} 아이템 {index} : {link}")

            detail_item_data = get_item_data(driver, link)
            item.update(detail_item_data)
            item_data.append(item)
            
            print(item)
        print(f'페이지 {i + 1} 완료')

    print("============================")
    print("결과")
    for item in item_data:
        print(item)

    df = pd.DataFrame(item_data)
    # 파일이 이미 있는지 체크
    if os.path.exists(csv_path):
        df.to_csv(csv_path, mode="a", index=False, header=False, encoding="utf-8", quoting=csv.QUOTE_ALL)
    else:
        df.to_csv(csv_path, mode="w", index=False, header=True, encoding="utf-8", quoting=csv.QUOTE_ALL)

키워드:  유모차 스토케
전체 아이템 수:  159
페이지수:  2
페이지 1: 89
페이지 1 아이템 0 : https://m.bunjang.co.kr/products/330635984?q=%EC%9C%A0%EB%AA%A8%EC%B0%A8%20%EC%8A%A4%ED%86%A0%EC%BC%80&ref=%EA%B2%80%EC%83%89%EA%B2%B0%EA%B3%BC
이미지 수: 6
다운로드 완료: C:/Potenup/SecondHanded-Strollers-PredictedPrice/data/raw/images/356fcf58-2382-46c5-b404-397e30c5e2a5/bungaejangter_356fcf58-2382-46c5-b404-397e30c5e2a5_0.jpg
다운로드 완료: C:/Potenup/SecondHanded-Strollers-PredictedPrice/data/raw/images/356fcf58-2382-46c5-b404-397e30c5e2a5/bungaejangter_356fcf58-2382-46c5-b404-397e30c5e2a5_1.jpg
다운로드 완료: C:/Potenup/SecondHanded-Strollers-PredictedPrice/data/raw/images/356fcf58-2382-46c5-b404-397e30c5e2a5/bungaejangter_356fcf58-2382-46c5-b404-397e30c5e2a5_2.jpg
다운로드 완료: C:/Potenup/SecondHanded-Strollers-PredictedPrice/data/raw/images/356fcf58-2382-46c5-b404-397e30c5e2a5/bungaejangter_356fcf58-2382-46c5-b404-397e30c5e2a5_3.jpg
다운로드 완료: C:/Potenup/SecondHanded-Strollers-PredictedPrice/data/raw/images/356fcf58-2382-46c5-b404-397e30c5e2a5/bu

## csv 데이터 확인

In [ ]:
df = pd.read_csv(r"C:\Potenup\SecondHanded-Strollers-PredictedPrice\data\\raw\\bungaejangter.csv", encoding="utf-8", engine="python", on_bad_lines="warn")

In [38]:
df.head(1)

,keyword,is_completed,uploaded_date,link,location,id,title,detail,condition,price,images_url
0,유모차 스토케,False,2025-08-20 07:43:46.595629,https://m.bunjang.co.kr/products/330635984?q=%...,NaN,356fcf58-2382-46c5-b404-397e30c5e2a5,스토케 트레일즈 유모차,출고가 150만원 이었습니다\n사진 그대로 구매시 30만원\n겨울용 패딩시트 제외시...,사용감 적음,200000.0,['https://media.bunjang.co.kr/product/33063598...


In [22]:
df.shape

(148, 11)

## is_completed 링크 문제 해결

In [ ]:
df.loc[df['is_completed']]

,keyword,is_completed,uploaded_date,link,location,id,title,detail,condition,price,images_url
146,유모차 스토케,True,2025-06-21 11:49:19.373481,https://m.bunjang.co.kr/products/336771288?q=%...,NaN,a5450f68-9021-4ead-ae57-9976c29d57e1,NaN,NaN,NaN,NaN,[]
147,유모차 스토케,True,2025-06-21 11:49:19.454824,https://m.bunjang.co.kr/products/338052842?q=%...,NaN,50eadce2-9fea-4a6e-b6c4-f7f1aa1692c0,NaN,NaN,NaN,NaN,[]


### 내부 아이템 데이터 다시 가져오기

In [ ]:
from file_util import ensure_dir
from image_util import download_image
import re

base_url = "https://m.bunjang.co.kr/"
data_base_path = "C:/Potenup/SecondHanded-Strollers-PredictedPrice/data/raw/"
image_base_path = "C:/Potenup/SecondHanded-Strollers-PredictedPrice/data/raw/images/"

for idx, row in df.loc[df['is_completed']].iterrows():
    print(row)
    uid = row['id']
    link = row['link']
    driver.get(link)

    time.sleep(2)

    item_data = get_item_data(driver, link)

    # 3. 해당 행 업데이트
    df.at[idx, 'title'] = item_data['title']
    df.at[idx, 'detail'] = item_data['detail']
    df.at[idx, 'condition'] = item_data['condition']
    df.at[idx, 'price'] = item_data['price']
    df.at[idx, 'images_url'] = item_data['images_url']

## location 처리

In [ ]:
from selenium.common.exceptions import NoSuchElementException

for idx, row in df[df["location"].isna()].iterrows():
    link = row["link"]
    driver.get(link)

    time.sleep(2)

    location_css = "#root > div > div > div.Productsstyle__Wrapper-sc-13cvfvh-0.eVEUVR > div.Productsstyle__ProductPageTop-sc-13cvfvh-1.WbLlq > div > div.Productsstyle__ProductContentWrapper-sc-13cvfvh-8.jGywBa > div > div.Productsstyle__ProductSummaryWrapper-sc-13cvfvh-11.iDkwQU > div > div:nth-child(1) > div:nth-child(2) > div:nth-child(2) > div:nth-child(5) > div.ProductSummarystyle__Value-sc-oxz0oy-21.eLyjky"
    try:
        location = driver.find_element(By.CSS_SELECTOR, location_css).text
    except NoSuchElementException:
        location = None
    
    df.at[idx, 'location'] = location

keyword                                                    유모차 스토케
is_completed                                                 False
uploaded_date                           2025-08-17 11:43:47.247596
link             https://m.bunjang.co.kr/products/351202942?q=%...
location                                                      None
id                            92eb1225-d0b3-4045-ae0f-add2b5b4d154
title                                          스토케 유모차 신생아패드 신생아시트
detail           스토케 익스플로리 유모차 V3 신생아패드 신생아시트 \n세탁해두었습니다  특별한 하...
condition                                                   사용감 적음
price                                                      12000.0
images_url       ['https://media.bunjang.co.kr/product/35120294...
Name: 7, dtype: object
keyword                                                    유모차 스토케
is_completed                                                 False
uploaded_date                           2025-08-17 11:43:47.351692
link             https://m.bunjang.co.k

In [55]:
df['location']

0                  서울특별시 용산구 한강로동
1                  서울특별시 용산구 한강로동
2                     세종특별자치시 보람동
3                   서울특별시 은평구 수색동
4                   대전광역시 중구 태평2동
                  ...            
143           전북특별자치도 전주시 완산구 중앙동
144               서울특별시 구로구 구로제5동
145               서울특별시 강동구 천호제2동
146    경기도 성남시 분당구 정자동\n정자역 1번 출구
147         대구광역시 중구 성내1동\n봉산문화회관
Name: location, Length: 148, dtype: object

### 세부 주소 삭제

In [56]:
df['location'] = df['location'].str.split("\n").str[0]

In [60]:
df.head(1)

,keyword,is_completed,uploaded_date,link,location,id,title,detail,condition,price,images_url
0,유모차 스토케,False,2025-08-20 07:43:46.595629,https://m.bunjang.co.kr/products/330635984?q=%...,서울특별시 용산구 한강로동,356fcf58-2382-46c5-b404-397e30c5e2a5,스토케 트레일즈 유모차,출고가 150만원 이었습니다\n사진 그대로 구매시 30만원\n겨울용 패딩시트 제외시...,사용감 적음,200000.0,['https://media.bunjang.co.kr/product/33063598...


In [ ]:
df.to_csv("C:\Potenup\SecondHanded-Strollers-PredictedPrice\data\preprocessed\\bungaejangter.csv")